In [37]:
import matplotlib.pyplot as plt
import vtk
import numpy as np
import sys
import math
import os
import glob
from vtk.util.numpy_support import *
import pandas
from multiprocessing import Pool
from scipy.stats import kurtosis, skew

In [38]:
def read_vti(filename):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()

def write_vti(filename,data):
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()

def compute_3d_to_1d_map(x,y,z,dimx,dimy,dimz):
    return x + dimx*(y+dimy*z)


def compare_moments(block_signature,feature):
    diff = 0
    weights = [0.25,0.25,0.25,0.25]
    for i in range(len(feature)):
        val = np.absolute(feature[i] - block_signature[i])
        #val = np.power(val,2.0)
        diff = diff + val*weights[i]
        
    return diff

In [40]:
## vortex data
xdim = 128
ydim = 128
zdim = 128

nbins = 64
initstep = 10
tsteps = 30
window = 10

xbox = 4
ybox = 4
zbox = 4

totPtsBlock = xbox*ybox*zbox

feature_dist_input = '../feature_dists/vortex_1.csv'

data_path = '/disk2/Source_Codes/multivar_sampling/Data/vortex_vti/vortex_'
varname = 'ImageScalars'
first_tstep = '/disk2/Source_Codes/multivar_sampling/Data/vortex_vti/vortex_10.vti'
out_file_path1 = '../out/vortex/classified_similarity_'
out_file_path2 = '../out/vortex/classified_temporal_'
out_file_path3 = '../out/vortex/pervoxel_similarity_'

In [29]:
# ## Pele data
# xdim = 100
# ydim = 50
# zdim = 100

# nbins = 64
# initstep = 0
# tsteps = 5
# window = 1

# xbox = 4
# ybox = 2
# zbox = 4
# totPtsBlock = xbox*ybox*zbox

# feature_dist_input = '../feature_dists/pele.csv'

# data_path = '/disk1/PeleData/Pele_resampled_vti/pele_resampled_'
# varname = 'density'
# first_tstep = '/disk1/PeleData/Pele_resampled_vti/pele_resampled_0.vti'
# out_file_path1 = '../out/pele/moment_similarity_'
# out_file_path2 = '../out/pele/classified_temporal_'
# out_file_path3 = '../out/pele/pervoxel_similarity_'

In [13]:
# ## mfix data
# xdim = 256
# ydim = 64
# zdim = 256

# nbins = 64
# initstep = 75
# tsteps = 408
# window = 1

# xbox = 4
# ybox = 4
# zbox = 4

# totPtsBlock = xbox*ybox*zbox

# feature_dist_input = '../feature_dists/mfix_bubble_datavals_2.csv'

# data_path = '/disk1/MFIX_bubble_fields_highres/original_timestep_'
# varname = 'ImageScalars'
# first_tstep = '/disk1/MFIX_bubble_fields/original_timestep_75.vti'
# out_file_path1 = '../out/mfix_case_3/moment_similarity_'
# out_file_path2 = '../out/mfix_case_3/classified_temporal_'
# out_file_path3 = '../out/mfix_case_3/pervoxel_similarity_'

In [41]:
## load feature distribution
df = pandas.read_csv(feature_dist_input)
feature_data = np.asarray(df[varname])

m1 = np.average(feature_data)
print 'mean is: ' + str(m1)

m2 = np.var(feature_data)
print 'var is: ' + str(m2)

m3 = skew(feature_data)
print 'skew is: ' + str(m3)

m4 = kurtosis(feature_data)
print 'kurtosis is: ' + str(m4)

feature = [m1,m2,m3,m4]

mean is: 7.309776470588234
var is: 2.5646487792618227
skew is: -0.65664699854
kurtosis is: -0.0542129760175


In [42]:
### compare with given feature using moments
for ii in range(initstep,tsteps,window):
    
    print 'processing tstep ' + str(ii)
    
    data_file = data_path + str(ii) + '.vti'
    
    data = read_vti(data_file)
    np_var_array = vtk.util.numpy_support.vtk_to_numpy(data.GetPointData().GetArray(varname))
    ## note that np_var_array has x and z swapped
    
    np_var_array = np.reshape(np_var_array,(zdim,ydim,xdim))
    classified_array = np.zeros_like(np_var_array)
    
    for k in range(0,zdim,zbox):
        for j in range(0,ydim,ybox):
            for i in range(0,xdim,xbox):
            
                ## note that np_var_array has x and z swapped. SO, k index in given first
                block_data = np_var_array[k:k+zbox, j:j+ybox, i:i+xbox] 
                #print k,k+zbox, j,j+ybox, i,i+xbox, block_data 
                block_data = block_data.reshape(totPtsBlock)
                
                block_signature = [np.average(block_data),np.var(block_data),skew(block_data),kurtosis(block_data)] 
                
                ## compare data
                comp_val = compare_moments(block_signature,feature)
                classified_array[k:k+zbox, j:j+ybox, i:i+xbox] = comp_val
                
                
    classified_array = classified_array.reshape(xdim*ydim*zdim)
    classified_array_vtk = vtk.util.numpy_support.numpy_to_vtk(classified_array)
    classified_array_vtk.SetName('moment_similarity')
    data.GetPointData().AddArray(classified_array_vtk)

    out_fname = out_file_path1 + str(ii) + '.vti'
    write_vti(out_fname,data)

processing tstep 10
processing tstep 20


In [69]:
### Per voxel neighborhood distribution matching instead of block-based comparison

neighborhood_x = 2
neighborhood_y = 2
neighborhood_z = 2

for ii in range(initstep,tsteps):
    
    print 'processing tstep ' + str(ii)
    
    data_file = data_path + str(ii) + '.vti'
    
    data = read_vti(data_file)
    np_var_array = vtk.util.numpy_support.vtk_to_numpy(data.GetPointData().GetArray(varname))
    ## note that np_var_array has x and z swapped
    
    np_var_array = np.reshape(np_var_array,(zdim,ydim,xdim))
    classified_array = np.zeros_like(np_var_array)
    
    ## do for each grid point
    for k in range(0,zdim):
        for j in range(0,ydim):
            for i in range(0,xdim):
            
                ## note that np_var_array has x and z swapped. SO, k index in given first
                
                if (k-neighborhood_z) >= 0:
                    kmin = (k-neighborhood_z)
                else:
                    kmin = 0
                
                if (k+neighborhood_z) <= zdim:
                    kmax = (k+neighborhood_z)
                else:
                    kmax = zdim
                    
                if (j-neighborhood_y) >= 0:
                    jmin = (j-neighborhood_y)
                else:
                    jmin = 0
                
                if (j+neighborhood_y) <= ydim:
                    jmax = (j+neighborhood_y)
                else:
                    jmax = ydim
                    
                if (i-neighborhood_x) >= 0:
                    imin = (i-neighborhood_x)
                else:
                    imin = 0
                
                if (i+neighborhood_x) <= xdim:
                    imax = (i+neighborhood_x)
                else:
                    imax = xdim    
                
                block_data = np_var_array[kmin:kmax, jmin:jmax, imin:imax] 
                
                totPtsBlock = np.shape(block_data)[0]*np.shape(block_data)[1]*np.shape(block_data)[2]
                
                block_data = block_data.reshape(totPtsBlock)
                
                block_signature = [np.average(block_data),np.var(block_data),skew(block_data),kurtosis(block_data)] 
                
                ## compare data
                comp_val = compare_moments(block_signature,feature)
                classified_array[k, j, i] = comp_val
                
                
    classified_array = classified_array.reshape(xdim*ydim*zdim)
    classified_array_vtk = vtk.util.numpy_support.numpy_to_vtk(classified_array)
    classified_array_vtk.SetName('feature_similarity')
    data.GetPointData().AddArray(classified_array_vtk)

    out_fname = out_file_path3 + str(ii) + '.vti'
    write_vti(out_fname,data)